In [1]:
import pandas as pd
import numpy as np
from pathlib import Path


In [2]:
df = pd.read_csv(r"../raw_data/data_house_full/companydata_part2.csv", dtype=str, low_memory=True)


In [3]:
uk_countries = ["ENGLAND", "SCOTLAND", "WALES", "NORTHERN IRELAND", "UNITED KINGDOM"]

In [4]:
df_uk = df[df["RegAddress.Country"].isin(uk_countries)]

In [5]:
df_uk.head()

,CompanyName,CompanyNumber,RegAddress.CareOf,RegAddress.POBox,RegAddress.AddressLine1,RegAddress.AddressLine2,RegAddress.PostTown,RegAddress.County,RegAddress.Country,RegAddress.PostCode,...,PreviousName_7.CONDATE,PreviousName_7.CompanyName,PreviousName_8.CONDATE,PreviousName_8.CompanyName,PreviousName_9.CONDATE,PreviousName_9.CompanyName,PreviousName_10.CONDATE,PreviousName_10.CompanyName,ConfStmtNextDueDate,ConfStmtLastMadeUpDate
3,BUCHANAN STAFFING GROUP HOLDINGS LIMITED,15360457,NaN,NaN,HATCHERS YARD,9 TANNER STREET,LONDON,NaN,UNITED KINGDOM,SE1 3L,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01/01/2026,18/12/2024
4,BUCHANAN STAFFING GROUP LIMITED,07710058,NaN,NaN,HATCHERS YARD,9 TANNER STREET,LONDON,NaN,UNITED KINGDOM,SE1 3LE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12/07/2026,28/06/2025
6,BUCHANAN STUDIO LTD,09740585,NaN,NaN,BUSPACE STUDIOS,CONLAN STREET,LONDON,NaN,ENGLAND,W10 5AP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15/08/2025,01/08/2024
7,BUCHANAN SUPPORT SERVICES LLP,SO303111,NaN,NaN,23 MONREITH ROAD,NEWLANDS,GLASGOW,NaN,SCOTLAND,G43 2NY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01/03/2026,15/02/2025
8,BUCHANAN SURVEYORS LTD,SC664467,NaN,NaN,19 WATERSLAP,FENWICK,KILMARNOCK,NaN,SCOTLAND,KA3 6AJ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30/06/2026,16/06/2025


In [6]:
df_uk.columns = df_uk.columns.str.strip()

In [7]:
# 2) Clean address + postcode
def clean_str(s): return s.fillna("").str.strip().str.upper()
def clean_postcode(s):
    pc = clean_str(s).str.replace(r"[^A-Z0-9]", "", regex=True)
    return np.where(pc.str.len() < 5, pc, pc.str[:-3] + " " + pc.str[-3:])

In [8]:
df_uk["AddressLine1"]   = clean_str(df_uk["RegAddress.AddressLine1"])
df_uk["PostCode_clean"] = clean_postcode(df_uk["RegAddress.PostCode"])

/tmp/ipykernel_171766/564625837.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_uk["AddressLine1"]   = clean_str(df_uk["RegAddress.AddressLine1"])
/tmp/ipykernel_171766/564625837.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_uk["PostCode_clean"] = clean_postcode(df_uk["RegAddress.PostCode"])


In [9]:
df_uk["AddressLine2"]   = clean_str(df_uk["RegAddress.AddressLine2"])

/tmp/ipykernel_171766/713273805.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_uk["AddressLine2"]   = clean_str(df_uk["RegAddress.AddressLine2"])


In [10]:
df_uk["Town_clean"]   = clean_str(df_uk.get("RegAddress.PostTown", pd.Series("", index=df_uk.index)))

/tmp/ipykernel_171766/1228941311.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_uk["Town_clean"]   = clean_str(df_uk.get("RegAddress.PostTown", pd.Series("", index=df_uk.index)))


In [11]:
# 2) Heuristic: choose the street-like line
#    - If AddressLine1 has street keywords, keep it.
#    - Else if AddressLine2 has them, use AddressLine2.
#    - Else fall back to AddressLine1.
STREET_WORDS = r"\b(ROAD|RD|STREET|ST|AVENUE|AVE|LANE|LN|DRIVE|DR|COURT|CT|WAY|PLACE|PL|SQUARE|SQ|CRESCENT|CRES|TERRACE|TER|GROVE|PARADE|HILL|HIGH\s+ST(REET)?|HIGH\s+RD|CLOSE|MEWS|ROW|QUAY|QUAYS|GATE|MARKET|MARKET\s+PLACE)\b"

l1_is_street = df_uk["AddressLine1"].str.contains(STREET_WORDS, regex=True, na=False)
l2_is_street = df_uk["AddressLine2"].str.contains(STREET_WORDS, regex=True, na=False)

df_uk["Address_street"] = np.where(
    l1_is_street, df_uk["AddressLine1"],
    np.where(l2_is_street, df_uk["AddressLine2"], df_uk["AddressLine1"])
)

# (Optional) build a display-friendly full address
df_uk["FullAddress"] = (
    df_uk["AddressLine1"]
      .str.cat(df_uk["AddressLine2"].mask(df_uk["AddressLine2"].eq("")), sep=", ", na_rep="")
      .str.strip(", ")
      .str.cat(df_uk["Town_clean"].mask(df_uk["Town_clean"].eq("")), sep=", ", na_rep="")
      .str.strip(", ")
      .str.cat(df_uk["PostCode_clean"].mask(df_uk["PostCode_clean"].eq("")), sep=", ", na_rep="")
      .str.strip(", ")
)


/tmp/ipykernel_171766/1380917566.py:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  l1_is_street = df_uk["AddressLine1"].str.contains(STREET_WORDS, regex=True, na=False)
/tmp/ipykernel_171766/1380917566.py:8: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  l2_is_street = df_uk["AddressLine2"].str.contains(STREET_WORDS, regex=True, na=False)
/tmp/ipykernel_171766/1380917566.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_uk["Address_street"] = np.where(
/tmp/ipykernel_171766/1380917566.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

In [12]:
# 3) Count unique companies per (postcode, chosen street address)
id_col = "CompanyNumber" if "CompanyNumber" in df_uk.columns else "CompanyName"

In [13]:
address_counts = (
    df_uk.groupby(["PostCode_clean", "Address_street"], dropna=False)[id_col]
         .nunique()
         .reset_index(name="Companies_at_Address")
         .sort_values("Companies_at_Address", ascending=False)
)

In [14]:
# 4) Postcode totals (for share)
postcode_counts = (
    df_uk.groupby("PostCode_clean", dropna=False)[id_col]
         .nunique()
         .reset_index(name="Companies_in_Postcode")
)

In [15]:
# 5) Merge + share; show heavy hitters (>=100)
compare = address_counts.merge(postcode_counts, on="PostCode_clean", how="left")
compare["Address_share_in_PC"] = (compare["Companies_at_Address"] / compare["Companies_in_Postcode"] * 100).round(1)

heavy_100plus = compare[compare["Companies_at_Address"] >= 100].copy() \
                        .sort_values(["Companies_at_Address","Companies_in_Postcode"], ascending=False)

heavy_100plus.info()

<class 'pandas.core.frame.DataFrame'>
Index: 175 entries, 0 to 174
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   PostCode_clean         175 non-null    object 
 1   Address_street         175 non-null    object 
 2   Companies_at_Address   175 non-null    int64  
 3   Companies_in_Postcode  175 non-null    int64  
 4   Address_share_in_PC    175 non-null    float64
dtypes: float64(1), int64(2), object(2)
memory usage: 8.2+ KB


In [16]:
heavy_100plus.head()

,PostCode_clean,Address_street,Companies_at_Address,Companies_in_Postcode,Address_share_in_PC
0,WC2H 9JQ,71-75 SHELTON STREET,11981,12152,98.6
1,EC1V 2NX,128 CITY ROAD,5947,9494,62.6
2,N1 7GU,20-22 WENLOCK ROAD,2774,5553,50.0
3,N1 7GU,20 WENLOCK ROAD,2704,5553,48.7
4,W1W 7LT,85 GREAT PORTLAND STREET,2449,2884,84.9


In [17]:
# 1) Build a one-row-per-(postcode, street) map with a single chosen FullAddress
def best_full(series: pd.Series) -> str:
    s = series.replace("", np.nan).dropna()
    if s.empty:
        return np.nan
    vc = s.value_counts()
    top_n = vc[vc.eq(vc.iloc[0])].index.tolist()  # all tied for most frequent
    # tie-breaker: prefer the one with more components (more commas), then longer length
    return max(top_n, key=lambda a: (a.count(","), len(a)))

fa_map = (
    df_uk.groupby(["PostCode_clean", "Address_street"])["FullAddress"]
         .agg(best_full)
         .reset_index()
         .rename(columns={"FullAddress": "FullAddress_best"})
)

# 2) Left-merge onto your heavy_100plus WITHOUT creating extra rows
heavy_100plus = heavy_100plus.merge(
    fa_map,
    on=["PostCode_clean", "Address_street"],
    how="left",
    validate="one_to_one"   # guarantees no row multiplication
)


In [18]:
# 3) Fallback if any are missing
heavy_100plus["FullAddress_best"] = heavy_100plus["FullAddress_best"].fillna(
    heavy_100plus["Address_street"] + ", " + heavy_100plus["PostCode_clean"]
)

heavy_100plus.head(10)

,PostCode_clean,Address_street,Companies_at_Address,Companies_in_Postcode,Address_share_in_PC,FullAddress_best
0,WC2H 9JQ,71-75 SHELTON STREET,11981,12152,98.6,"71-75 SHELTON STREET, COVENT GARDEN, LONDON, W..."
1,EC1V 2NX,128 CITY ROAD,5947,9494,62.6,"128 CITY ROAD, LONDON, EC1V 2NX"
2,N1 7GU,20-22 WENLOCK ROAD,2774,5553,50.0,"20-22 WENLOCK ROAD, LONDON, N1 7GU"
3,N1 7GU,20 WENLOCK ROAD,2704,5553,48.7,"20 WENLOCK ROAD, LONDON, N1 7GU"
4,W1W 7LT,85 GREAT PORTLAND STREET,2449,2884,84.9,"85 GREAT PORTLAND STREET, FIRST FLOOR, LONDON,..."
5,WC1N 3AX,27 OLD GLOUCESTER STREET,2256,2314,97.5,"27 OLD GLOUCESTER STREET, LONDON, WC1N 3AX"
6,EC1V 2NX,124 CITY ROAD,2142,9494,22.6,"124 CITY ROAD, LONDON, EC1V 2NX"
7,EC2A 4NE,PAUL STREET,2024,4383,46.2,"3RD FLOOR, 86-90, PAUL STREET, LONDON, EC2A 4NE"
8,EC2A 4NE,86-90 PAUL STREET,2008,4383,45.8,"86-90 PAUL STREET, LONDON, EC2A 4NE"
9,W1W 5PF,167-169 GREAT PORTLAND STREET,1215,1564,77.7,"167-169 GREAT PORTLAND STREET, 5TH FLOOR, LOND..."


In [19]:
heavy_100plus


,PostCode_clean,Address_street,Companies_at_Address,Companies_in_Postcode,Address_share_in_PC,FullAddress_best
0,WC2H 9JQ,71-75 SHELTON STREET,11981,12152,98.6,"71-75 SHELTON STREET, COVENT GARDEN, LONDON, W..."
1,EC1V 2NX,128 CITY ROAD,5947,9494,62.6,"128 CITY ROAD, LONDON, EC1V 2NX"
2,N1 7GU,20-22 WENLOCK ROAD,2774,5553,50.0,"20-22 WENLOCK ROAD, LONDON, N1 7GU"
3,N1 7GU,20 WENLOCK ROAD,2704,5553,48.7,"20 WENLOCK ROAD, LONDON, N1 7GU"
4,W1W 7LT,85 GREAT PORTLAND STREET,2449,2884,84.9,"85 GREAT PORTLAND STREET, FIRST FLOOR, LONDON,..."
...,...,...,...,...,...,...
170,TA1 2PX,BLACKBROOK PARK AVENUE,100,146,68.5,"GOODWOOD HOUSE, BLACKBROOK PARK AVENUE, TAUNTO..."
171,OX28 4BE,FIRST FLOOR 1 DES ROCHES SQUARE,100,115,87.0,"FIRST FLOOR 1 DES ROCHES SQUARE, WITAN WAY, WI..."
172,CM12 0DU,RADFORD CRESCENT,100,112,89.3,"UNIT 7A, RADFORD CRESCENT, BILLERICAY, CM12 0DU"
173,WA4 2QL,207 KNUTSFORD ROAD,100,103,97.1,"207 KNUTSFORD ROAD, GRAPPENHALL, WARRINGTON, W..."


In [20]:
# Shows addresses in the same postcode that look similar (ignoring numbers/ranges)
def candidate_groups(heavy_tbl, min_companies=100, postcode=None):
    df = heavy_tbl[heavy_tbl["Companies_at_Address"] >= min_companies].copy()
    if postcode:
        df = df[df["PostCode_clean"].eq(postcode)]
    base = (df["Address_street"]
              .str.replace(r"\b\d+(?:-\d+)?\b", "", regex=True)
              .str.replace(r"[^\w\s]", "", regex=True)
              .str.replace(r"\s+", " ", regex=True)
              .str.strip())
    cand = df.assign(_base=base).groupby(["PostCode_clean","_base"]) \
               .filter(lambda g: g["Address_street"].nunique()>1)
    return cand.sort_values(["PostCode_clean","_base","Companies_at_Address"], ascending=[True,True,False])

# Example:
# candidate_groups(heavy_100plus, postcode="N1 7GU").head(20)


In [21]:
import re
import numpy as np
import pandas as pd

# Normalizer for comparing streets: drop number/ranges, unit/floor words, punctuation, extra spaces
def _norm_street(s: pd.Series) -> pd.Series:
    s = s.fillna("").str.upper()
    s = s.str.replace(r"\b\d+(?:\s*-\s*\d+)?\b", "", regex=True)  # remove numbers & ranges
    s = s.str.replace(r"\b(FLAT|UNIT|SUITE|APT|APARTMENT|ROOM|LEVEL|FLOOR|OFFICE)\b", "", regex=True)
    s = s.str.replace(r"[^\w\s]", "", regex=True)                 # punctuation
    s = s.str.replace(r"\s+", " ", regex=True).str.strip()
    return s

postcodes_with_issue = []

for pc, sub in heavy_100plus.groupby("PostCode_clean"):
    base = _norm_street(sub["Address_street"])
    # if the normalized base repeats across different Address_street values -> potential merge needed
    dup_mask = base.duplicated(keep=False)
    if dup_mask.any():
        postcodes_with_issue.append(pc)

# Unique + sorted list
postcodes_with_issue = sorted(set(postcodes_with_issue))

# (optional) quick peek
len(postcodes_with_issue), postcodes_with_issue[:10]


(5, ['EC1V 2NX', 'EC2A 4NE', 'HA4 7AE', 'IP28 7DE', 'N1 7GU'])

In [22]:
MERGES = [
    ("N1 7GU", "20 WENLOCK ROAD", "20-22 WENLOCK ROAD")
    # add more lines later if you want
]

In [23]:
# make a working column (keeps your originals intact)
df_uk["Address_street_merged"] = df_uk["Address_street"]

for pc, frm, to in MERGES:
    mask = df_uk["PostCode_clean"].eq(pc) & df_uk["Address_street"].eq(frm)
    df_uk.loc[mask, "Address_street_merged"] = to


/tmp/ipykernel_171766/3152388706.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_uk["Address_street_merged"] = df_uk["Address_street"]


In [24]:
id_col = "CompanyNumber" if "CompanyNumber" in df_uk.columns else "CompanyName"

address_counts_m = (
    df_uk.groupby(["PostCode_clean","Address_street_merged"])[id_col]
         .nunique()
         .reset_index(name="Companies_at_Address")
)
postcode_counts = (
    df_uk.groupby("PostCode_clean")[id_col]
         .nunique()
         .reset_index(name="Companies_in_Postcode")
)
# one representative FullAddress per merged group
fa_map = (
    df_uk.groupby(["PostCode_clean","Address_street_merged"])["FullAddress"]
         .agg(lambda s: s.mode().iat[0] if not s.mode().empty else s.dropna().iloc[0] if s.notna().any() else None)
         .reset_index(name="FullAddress_best")
)

heavy_m = (address_counts_m
           .merge(postcode_counts, on="PostCode_clean", how="left")
           .merge(fa_map, on=["PostCode_clean","Address_street_merged"], how="left"))
heavy_m["Address_share_in_PC"] = (heavy_m["Companies_at_Address"] / heavy_m["Companies_in_Postcode"] * 100).round(1)

heavy_100plus_merged = (heavy_m[heavy_m["Companies_at_Address"] >= 100]
                        .sort_values(["Companies_at_Address","Companies_in_Postcode"], ascending=False)
                        .rename(columns={"Address_street_merged":"Address_street"}))

# See the result for N1 7GU
heavy_100plus_merged.query('PostCode_clean == "N1 7GU"')


,PostCode_clean,Address_street,Companies_at_Address,Companies_in_Postcode,FullAddress_best,Address_share_in_PC
219432,N1 7GU,20-22 WENLOCK ROAD,5478,5553,"20-22 WENLOCK ROAD, LONDON, N1 7GU",98.6


In [25]:
# See the result for EC1V 2NX
heavy_100plus_merged.query('PostCode_clean == "EC1V 2NX"')

,PostCode_clean,Address_street,Companies_at_Address,Companies_in_Postcode,FullAddress_best,Address_share_in_PC
116162,EC1V 2NX,128 CITY ROAD,5947,9494,"128 CITY ROAD, LONDON, EC1V 2NX",62.6
116131,EC1V 2NX,124 CITY ROAD,2142,9494,"124 CITY ROAD, LONDON, EC1V 2NX",22.6
116153,EC1V 2NX,124-128 CITY ROAD,545,9494,"124-128 CITY ROAD, LONDON, EC1V 2NX",5.7
116202,EC1V 2NX,CITY ROAD,186,9494,"KEMP HOUSE 152-160, CITY ROAD, LONDON, EC1V 2NX",2.0
116185,EC1V 2NX,160 CITY ROAD,182,9494,"KEMP HOUSE, 160 CITY ROAD, LONDON, EC1V 2NX",1.9
116125,EC1V 2NX,124 CITY ROAD,144,9494,"124 CITY ROAD, LONDON, EC1V 2NX",1.5


In [26]:
MERGES = [
    ("N1 7GU",  "20 WENLOCK ROAD",  "20-22 WENLOCK ROAD"),
    ("EC1V 2NX","124 CITY ROAD",    "124-128 CITY ROAD"),
    ("EC1V 2NX","CITY ROAD",        "152-160 CITY ROAD"),  # this row's FullAddress shows “KEMP HOUSE 152-160…”
    ("EC1V 2NX","160 CITY ROAD",    "152-160 CITY ROAD"),
]

In [27]:
df_uk["Address_street_merged"] = df_uk["Address_street"]
for pc, frm, to in MERGES:
    mask = df_uk["PostCode_clean"].eq(pc) & df_uk["Address_street"].eq(frm)
    df_uk.loc[mask, "Address_street_merged"] = to


/tmp/ipykernel_171766/863531806.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_uk["Address_street_merged"] = df_uk["Address_street"]


In [28]:
df_uk["Address_street_merged"] = df_uk["Address_street"]
for pc, frm, to in MERGES:
    mask = df_uk["PostCode_clean"].eq(pc) & df_uk["Address_street"].eq(frm)
    df_uk.loc[mask, "Address_street_merged"] = to


/tmp/ipykernel_171766/863531806.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_uk["Address_street_merged"] = df_uk["Address_street"]


In [29]:
heavy_100plus_merged.head(10)

,PostCode_clean,Address_street,Companies_at_Address,Companies_in_Postcode,FullAddress_best,Address_share_in_PC
375786,WC2H 9JQ,71-75 SHELTON STREET,11981,12152,"71-75 SHELTON STREET, COVENT GARDEN, LONDON, W...",98.6
116162,EC1V 2NX,128 CITY ROAD,5947,9494,"128 CITY ROAD, LONDON, EC1V 2NX",62.6
219432,N1 7GU,20-22 WENLOCK ROAD,5478,5553,"20-22 WENLOCK ROAD, LONDON, N1 7GU",98.6
367616,W1W 7LT,85 GREAT PORTLAND STREET,2449,2884,"85 GREAT PORTLAND STREET, FIRST FLOOR, LONDON,...",84.9
375056,WC1N 3AX,27 OLD GLOUCESTER STREET,2256,2314,"27 OLD GLOUCESTER STREET, LONDON, WC1N 3AX",97.5
116131,EC1V 2NX,124 CITY ROAD,2142,9494,"124 CITY ROAD, LONDON, EC1V 2NX",22.6
116730,EC2A 4NE,PAUL STREET,2024,4383,"3RD FLOOR, 86-90, PAUL STREET, LONDON, EC2A 4NE",46.2
116702,EC2A 4NE,86-90 PAUL STREET,2008,4383,"86-90 PAUL STREET, LONDON, EC2A 4NE",45.8
367421,W1W 5PF,167-169 GREAT PORTLAND STREET,1215,1564,"167-169 GREAT PORTLAND STREET, 5TH FLOOR, LOND...",77.7
375424,WC2A 2JR,7 BELL YARD,1124,1197,"7 BELL YARD, LONDON, WC2A 2JR",93.9


In [30]:
heavy_100plus_merged.query('PostCode_clean == "EC2A 4NE"')

,PostCode_clean,Address_street,Companies_at_Address,Companies_in_Postcode,FullAddress_best,Address_share_in_PC
116730,EC2A 4NE,PAUL STREET,2024,4383,"3RD FLOOR, 86-90, PAUL STREET, LONDON, EC2A 4NE",46.2
116702,EC2A 4NE,86-90 PAUL STREET,2008,4383,"86-90 PAUL STREET, LONDON, EC2A 4NE",45.8
116686,EC2A 4NE,86 - 90 PAUL STREET,236,4383,"3RD FLOOR, 86 - 90 PAUL STREET, LONDON, EC2A 4NE",5.4


In [31]:
heavy_100plus_merged.query('PostCode_clean == "HA4 7AE"')

,PostCode_clean,Address_street,Companies_at_Address,Companies_in_Postcode,FullAddress_best,Address_share_in_PC
148486,HA4 7AE,"17 KING EDWARDS ROAD,",556,707,"2ND FLOOR COLLEGE HOUSE, 17 KING EDWARDS ROAD,...",78.6
148485,HA4 7AE,17 KING EDWARDS ROAD,126,707,"2ND FLOOR COLLEGE HOUSE, 17 KING EDWARDS ROAD,...",17.8


In [32]:
heavy_100plus_merged.query('PostCode_clean == "IP28 7DE"')

,PostCode_clean,Address_street,Companies_at_Address,Companies_in_Postcode,FullAddress_best,Address_share_in_PC
167646,IP28 7DE,82A JAMES CARTER ROAD,524,1989,"82A JAMES CARTER ROAD, MILDENHALL, IP28 7DE",26.3
167620,IP28 7DE,82 JAMES CARTER ROAD,469,1989,"UNIT A, 82 JAMES CARTER ROAD, MILDENHALL, IP28...",23.6
167731,IP28 7DE,UNIT 82A JAMES CARTER ROAD,284,1989,"UNIT 82A JAMES CARTER ROAD, MILDENHALL, BURY S...",14.3
167691,IP28 7DE,JAMES CARTER ROAD,148,1989,"UNIT A 82, JAMES CARTER ROAD, MILDENHALL, IP28...",7.4
167738,IP28 7DE,UNIT A 82 JAMES CARTER ROAD,135,1989,"UNIT A 82 JAMES CARTER ROAD, MILDENHALL, BURY ...",6.8
167710,IP28 7DE,SUITE A 82 JAMES CARTER ROAD,117,1989,"SUITE A 82 JAMES CARTER ROAD, MILDENHALL, BURY...",5.9


len(postcodes_with_issue), postcodes_with_issue[:10]


Merges need to be added higher up, before we actually create the merged_address

In [33]:
MERGES = [
    # N1 7GU → one row
    ("N1 7GU",  "20 WENLOCK ROAD",            "20-22 WENLOCK ROAD"),

    # EC1V 2NX → keep ONLY two rows: 124–128 and 152–160
    ("EC1V 2NX","124 CITY ROAD",              "124-128 CITY ROAD"),
    ("EC1V 2NX","128 CITY ROAD",              "124-128 CITY ROAD"),
    ("EC1V 2NX","CITY ROAD",                  "152-160 CITY ROAD"),
    ("EC1V 2NX","160 CITY ROAD",              "152-160 CITY ROAD"),

    # EC2A 4NE → one row
    ("EC2A 4NE","PAUL STREET",                "86-90 PAUL STREET"),
    ("EC2A 4NE","86 - 90 PAUL STREET",        "86-90 PAUL STREET"),

    # HA4 7AE → one row
    ("HA4 7AE","17 KING EDWARDS ROAD,",       "17 KING EDWARDS ROAD"),

    # IP28 7DE → one row: canonical "82 JAMES CARTER ROAD"
    ("IP28 7DE","UNIT 82A JAMES CARTER ROAD", "82 JAMES CARTER ROAD"),
    ("IP28 7DE","UNIT A 82 JAMES CARTER ROAD","82 JAMES CARTER ROAD"),
    ("IP28 7DE","SUITE A 82 JAMES CARTER ROAD","82 JAMES CARTER ROAD"),
    ("IP28 7DE","82A JAMES CARTER ROAD",      "82 JAMES CARTER ROAD"),
    ("IP28 7DE","JAMES CARTER ROAD",          "82 JAMES CARTER ROAD"),
]


In [34]:
id_col = "CompanyNumber" if "CompanyNumber" in df_uk.columns else "CompanyName"

address_counts_m = (
    df_uk.groupby(["PostCode_clean","Address_street_merged"])[id_col]
         .nunique()
         .reset_index(name="Companies_at_Address")
)
postcode_counts = (
    df_uk.groupby("PostCode_clean")[id_col]
         .nunique()
         .reset_index(name="Companies_in_Postcode")
)
# one representative FullAddress per merged group
fa_map = (
    df_uk.groupby(["PostCode_clean","Address_street_merged"])["FullAddress"]
         .agg(lambda s: s.mode().iat[0] if not s.mode().empty else s.dropna().iloc[0] if s.notna().any() else None)
         .reset_index(name="FullAddress_best")
)

heavy_m = (address_counts_m
           .merge(postcode_counts, on="PostCode_clean", how="left")
           .merge(fa_map, on=["PostCode_clean","Address_street_merged"], how="left"))
heavy_m["Address_share_in_PC"] = (heavy_m["Companies_at_Address"] / heavy_m["Companies_in_Postcode"] * 100).round(1)

heavy_100plus_merged = (heavy_m[heavy_m["Companies_at_Address"] >= 100]
                        .sort_values(["Companies_at_Address","Companies_in_Postcode"], ascending=False)
                        .rename(columns={"Address_street_merged":"Address_street"}))

In [35]:
heavy_100plus_merged.query('PostCode_clean in ["N1 7GU","EC1V 2NX","EC2A 4NE","HA4 7AE","IP28 7DE"]') \
    .sort_values(["PostCode_clean","Companies_at_Address"], ascending=[True,False])


,PostCode_clean,Address_street,Companies_at_Address,Companies_in_Postcode,FullAddress_best,Address_share_in_PC
116161,EC1V 2NX,128 CITY ROAD,5947,9494,"128 CITY ROAD, LONDON, EC1V 2NX",62.6
116152,EC1V 2NX,124-128 CITY ROAD,2687,9494,"124 CITY ROAD, LONDON, EC1V 2NX",28.3
116180,EC1V 2NX,152-160 CITY ROAD,430,9494,"KEMP HOUSE, 160 CITY ROAD, LONDON, EC1V 2NX",4.5
116125,EC1V 2NX,124 CITY ROAD,144,9494,"124 CITY ROAD, LONDON, EC1V 2NX",1.5
116727,EC2A 4NE,PAUL STREET,2024,4383,"3RD FLOOR, 86-90, PAUL STREET, LONDON, EC2A 4NE",46.2
116699,EC2A 4NE,86-90 PAUL STREET,2008,4383,"86-90 PAUL STREET, LONDON, EC2A 4NE",45.8
116683,EC2A 4NE,86 - 90 PAUL STREET,236,4383,"3RD FLOOR, 86 - 90 PAUL STREET, LONDON, EC2A 4NE",5.4
148483,HA4 7AE,"17 KING EDWARDS ROAD,",556,707,"2ND FLOOR COLLEGE HOUSE, 17 KING EDWARDS ROAD,...",78.6
148482,HA4 7AE,17 KING EDWARDS ROAD,126,707,"2ND FLOOR COLLEGE HOUSE, 17 KING EDWARDS ROAD,...",17.8
167643,IP28 7DE,82A JAMES CARTER ROAD,524,1989,"82A JAMES CARTER ROAD, MILDENHALL, IP28 7DE",26.3


In [36]:
# Normalize spacing and hyphens on the working key
df_uk["Address_street_merged"] = (
    df_uk["Address_street_merged"]
      .str.replace(r"\s*-\s*", "-", regex=True)   # "124 - 128" -> "124-128"
      .str.replace(r"\s+", " ", regex=True)       # collapse double spaces
      .str.strip()
)

# Make sure EC1V 2NX '124 CITY ROAD' is merged into '124-128 CITY ROAD'
df_uk.loc[
    (df_uk["PostCode_clean"].eq("EC1V 2NX")) &
    (df_uk["Address_street_merged"].str.fullmatch(r"124\s+CITY\s+ROAD")),
    "Address_street_merged"
] = "124-128 CITY ROAD"


/tmp/ipykernel_171766/1296275775.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_uk["Address_street_merged"] = (


In [37]:
id_col = "CompanyNumber" if "CompanyNumber" in df_uk.columns else "CompanyName"

address_counts_m = (
    df_uk.groupby(["PostCode_clean","Address_street_merged"], dropna=False)[id_col]
         .nunique()
         .reset_index(name="Companies_at_Address")
)
postcode_counts = (
    df_uk.groupby("PostCode_clean", dropna=False)[id_col]
         .nunique()
         .reset_index(name="Companies_in_Postcode")
)

def best_full(s):
    s = s.dropna()
    if s.empty: return None
    m = s.mode()
    return m.iat[0] if not m.empty else s.iloc[0]

fa_map = (
    df_uk.groupby(["PostCode_clean","Address_street_merged"])["FullAddress"]
         .agg(best_full)
         .reset_index()
         .rename(columns={"FullAddress":"FullAddress_best"})
)

heavy_m = (address_counts_m
           .merge(postcode_counts, on="PostCode_clean", how="left")
           .merge(fa_map, on=["PostCode_clean","Address_street_merged"], how="left"))
heavy_m["Address_share_in_PC"] = (
    heavy_m["Companies_at_Address"] / heavy_m["Companies_in_Postcode"] * 100
).round(1)

heavy_100plus_merged = (
    heavy_m[heavy_m["Companies_at_Address"] >= 100]
      .sort_values(["Companies_at_Address","Companies_in_Postcode"], ascending=False)
      .rename(columns={"Address_street_merged":"Address_street"})
)


In [38]:
heavy_100plus_merged.query('PostCode_clean == "EC1V 2NX"')


,PostCode_clean,Address_street,Companies_at_Address,Companies_in_Postcode,FullAddress_best,Address_share_in_PC
115963,EC1V 2NX,128 CITY ROAD,5947,9494,"128 CITY ROAD, LONDON, EC1V 2NX",62.6
115954,EC1V 2NX,124-128 CITY ROAD,2837,9494,"124 CITY ROAD, LONDON, EC1V 2NX",29.9
115979,EC1V 2NX,152-160 CITY ROAD,461,9494,"KEMP HOUSE, 160 CITY ROAD, LONDON, EC1V 2NX",4.9


In [39]:
heavy_100plus_merged.info()

<class 'pandas.core.frame.DataFrame'>
Index: 172 entries, 375097 to 140553
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   PostCode_clean         172 non-null    object 
 1   Address_street         172 non-null    object 
 2   Companies_at_Address   172 non-null    int64  
 3   Companies_in_Postcode  172 non-null    int64  
 4   FullAddress_best       172 non-null    object 
 5   Address_share_in_PC    172 non-null    float64
dtypes: float64(1), int64(2), object(3)
memory usage: 9.4+ KB


In [40]:
pd.set_option('display.max_rows', 500)
heavy_100plus_merged.sort_values('PostCode_clean')['PostCode_clean'].value_counts().head()

PostCode_clean
IP28 7DE    6
EC1V 2NX    3
BH16 6FA    2
HA4 7AE     2
EC2A 4NE    2
Name: count, dtype: int64

In [41]:
heavy_100plus_merged.sort_values('PostCode_clean').head(3)

,PostCode_clean,Address_street,Companies_at_Address,Companies_in_Postcode,FullAddress_best,Address_share_in_PC
15725,B90 8AH,LUMANERI HOUSE BLYTHE GATE,127,141,"LUMANERI HOUSE BLYTHE GATE, BLYTHE VALLEY PARK...",90.1
26117,BH16 6FA,UNIT 13 FREELAND PARK WAREHAM ROAD,134,543,"UNIT 13 FREELAND PARK WAREHAM ROAD, LYTCHETT M...",24.7
26120,BH16 6FA,WAREHAM ROAD,233,543,"LYTCHETT HOUSE 13 FREELAND PARK, WAREHAM ROAD,...",42.9


In [42]:
heavy_100plus_merged.query('PostCode_clean == "BH16 6FA"')

,PostCode_clean,Address_street,Companies_at_Address,Companies_in_Postcode,FullAddress_best,Address_share_in_PC
26120,BH16 6FA,WAREHAM ROAD,233,543,"LYTCHETT HOUSE 13 FREELAND PARK, WAREHAM ROAD,...",42.9
26117,BH16 6FA,UNIT 13 FREELAND PARK WAREHAM ROAD,134,543,"UNIT 13 FREELAND PARK WAREHAM ROAD, LYTCHETT M...",24.7


In [43]:
heavy_100plus_merged.query('PostCode_clean == "W1W 7LT"')

,PostCode_clean,Address_street,Companies_at_Address,Companies_in_Postcode,FullAddress_best,Address_share_in_PC
366963,W1W 7LT,85 GREAT PORTLAND STREET,2449,2884,"85 GREAT PORTLAND STREET, FIRST FLOOR, LONDON,...",84.9
366979,W1W 7LT,"85 GREAT PORTLAND STREET, FIRST FLOOR",209,2884,"85 GREAT PORTLAND STREET, FIRST FLOOR, LONDON,...",7.2


In [44]:
heavy_100plus_merged.query('PostCode_clean == "WA1 1RL"')

,PostCode_clean,Address_street,Companies_at_Address,Companies_in_Postcode,FullAddress_best,Address_share_in_PC
370444,WA1 1RL,840 IBIS COURT,305,469,"840 IBIS COURT, CENTRE PARK, WARRINGTON, WA1 1RL",65.0


In [45]:
# Canonicals:
# BH16 6FA  -> "13 FREELAND PARK"
# W1W 7LT   -> "85 GREAT PORTLAND STREET"
# WA1 1RL   -> "840 IBIS COURT"

MERGES += [
    # BH16 6FA (collapse all to "13 FREELAND PARK")
    ("BH16 6FA", "UNIT 13 FREELAND PARK WAREHAM ROAD", "13 FREELAND PARK"),
    ("BH16 6FA", "13 FREELAND PARK WAREHAM ROAD",      "13 FREELAND PARK"),
    ("BH16 6FA", "WAREHAM ROAD",                       "13 FREELAND PARK"),

    # W1W 7LT (drop floor suffix -> canonical)
    ("W1W 7LT",  "85 GREAT PORTLAND STREET, FIRST FLOOR", "85 GREAT PORTLAND STREET"),
    ("W1W 7LT",  "85 GREAT PORTLAND STREET FIRST FLOOR",  "85 GREAT PORTLAND STREET"),  # safety variant

    # WA1 1RL (brand -> base address)
    ("WA1 1RL",  "OPTIONIS HOUSE 840 IBIS COURT", "840 IBIS COURT"),
]


In [46]:
# apply
df_uk["Address_street_merged"] = df_uk.get("Address_street_merged", df_uk["Address_street"])
for pc, frm, to in MERGES:
    mask = df_uk["PostCode_clean"].eq(pc) & df_uk["Address_street_merged"].eq(frm)
    df_uk.loc[mask, "Address_street_merged"] = to

# rebuild heavy_m / heavy_100plus_merged (same code you already have)


/tmp/ipykernel_171766/441232984.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_uk["Address_street_merged"] = df_uk.get("Address_street_merged", df_uk["Address_street"])


In [47]:
import pandas as pd, numpy as np
from pathlib import Path

# --- 1) Apply your MERGES to the working key
df_uk["Address_street_merged"] = df_uk.get("Address_street_merged", df_uk["Address_street"])

for pc, frm, to in MERGES:
    mask = df_uk["PostCode_clean"].eq(pc) & df_uk["Address_street_merged"].eq(frm)
    df_uk.loc[mask, "Address_street_merged"] = to

# normalize spacing/hyphens (helps catch tiny variants)
df_uk["Address_street_merged"] = (
    df_uk["Address_street_merged"].str.replace(r"\s*-\s*", "-", regex=True)
                                   .str.replace(r"\s+", " ", regex=True)
                                   .str.strip()
)

# --- 2) Recompute heavy table on merged key
id_col = "CompanyNumber" if "CompanyNumber" in df_uk.columns else "CompanyName"

address_counts_m = (
    df_uk.groupby(["PostCode_clean","Address_street_merged"], dropna=False)[id_col]
         .nunique()
         .reset_index(name="Companies_at_Address")
)
postcode_counts = (
    df_uk.groupby("PostCode_clean", dropna=False)[id_col]
         .nunique()
         .reset_index(name="Companies_in_Postcode")
)

# one representative FullAddress per group
def best_full(s: pd.Series):
    s = s.dropna()
    if s.empty: return None
    m = s.mode()
    return m.iat[0] if not m.empty else s.iloc[0]

fa_map = (
    df_uk.groupby(["PostCode_clean","Address_street_merged"])["FullAddress"]
         .agg(best_full)
         .reset_index()
         .rename(columns={"FullAddress":"FullAddress_best"})
)

heavy_m = (address_counts_m
           .merge(postcode_counts, on="PostCode_clean", how="left")
           .merge(fa_map, on=["PostCode_clean","Address_street_merged"], how="left"))
heavy_m["Address_share_in_PC"] = (
    heavy_m["Companies_at_Address"] / heavy_m["Companies_in_Postcode"] * 100
).round(1)

# --- 3) Final ≥100 list
heavy_100plus_merged = (
    heavy_m[heavy_m["Companies_at_Address"] >= 100]
      .sort_values(["Companies_at_Address","Companies_in_Postcode"], ascending=False)
      .rename(columns={"Address_street_merged":"Address_street"})
)

# --- 4) Save
OUT = Path("../raw_data/data_house_full/processed"); OUT.mkdir(parents=True, exist_ok=True)
heavy_m.to_csv(OUT / "heavy_hitters_all_merged.csv", index=False)
heavy_100plus_merged.to_csv(OUT / "heavy_hitters_100plus_merged.csv", index=False)

print(
    f"Unique merged addresses: {len(heavy_m):,} | "
    f'≥100 companies: {len(heavy_100plus_merged):,} | '
    f"Saved to: {OUT}"
)

display(heavy_100plus_merged.head(25))


/tmp/ipykernel_171766/391217320.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_uk["Address_street_merged"] = df_uk.get("Address_street_merged", df_uk["Address_street"])
/tmp/ipykernel_171766/391217320.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_uk["Address_street_merged"] = (


Unique merged addresses: 390,568 | ≥100 companies: 162 | Saved to: ../raw_data/data_house_full/processed


,PostCode_clean,Address_street,Companies_at_Address,Companies_in_Postcode,FullAddress_best,Address_share_in_PC
375084,WC2H 9JQ,71-75 SHELTON STREET,12015,12152,"71-75 SHELTON STREET, COVENT GARDEN, LONDON, W...",98.9
115952,EC1V 2NX,124-128 CITY ROAD,8784,9494,"128 CITY ROAD, LONDON, EC1V 2NX",92.5
219032,N1 7GU,20-22 WENLOCK ROAD,5511,5553,"20-22 WENLOCK ROAD, LONDON, N1 7GU",99.2
116488,EC2A 4NE,86-90 PAUL STREET,4273,4383,"3RD FLOOR, 86-90, PAUL STREET, LONDON, EC2A 4NE",97.5
366953,W1W 7LT,85 GREAT PORTLAND STREET,2676,2884,"85 GREAT PORTLAND STREET, FIRST FLOOR, LONDON,...",92.8
374365,WC1N 3AX,27 OLD GLOUCESTER STREET,2256,2314,"27 OLD GLOUCESTER STREET, LONDON, WC1N 3AX",97.5
167297,IP28 7DE,82 JAMES CARTER ROAD,1681,1989,"UNIT 82A JAMES CARTER ROAD, MILDENHALL, BURY S...",84.5
366764,W1W 5PF,167-169 GREAT PORTLAND STREET,1289,1564,"167-169 GREAT PORTLAND STREET, 5TH FLOOR, LOND...",82.4
374728,WC2A 2JR,7 BELL YARD,1124,1197,"7 BELL YARD, LONDON, WC2A 2JR",93.9
370426,WA1 1RG,320 FIRECREST COURT,891,914,"320 FIRECREST COURT, CENTRE PARK, WARRINGTON, ...",97.5


In [48]:
heavy_100plus_merged.info()

<class 'pandas.core.frame.DataFrame'>
Index: 162 entries, 375084 to 140549
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   PostCode_clean         162 non-null    object 
 1   Address_street         162 non-null    object 
 2   Companies_at_Address   162 non-null    int64  
 3   Companies_in_Postcode  162 non-null    int64  
 4   FullAddress_best       162 non-null    object 
 5   Address_share_in_PC    162 non-null    float64
dtypes: float64(1), int64(2), object(3)
memory usage: 8.9+ KB


In [49]:
heavy_100plus_merged.sort_values('PostCode_clean')['PostCode_clean'].value_counts().head()

PostCode_clean
PO8 0BT     2
EC1V 2NX    2
B90 8AH     1
BH16 6FA    1
BL6 6HG     1
Name: count, dtype: int64

In [50]:
heavy_100plus_merged.query('PostCode_clean == "PO8 0BT"')

,PostCode_clean,Address_street,Companies_at_Address,Companies_in_Postcode,FullAddress_best,Address_share_in_PC
272024,PO8 0BT,ENTERPRISE ROAD,150,278,"UNIT 29 HIGHCROFT INDUSTRIAL ESTATE, ENTERPRIS...",54.0
272031,PO8 0BT,UNIT 29 HIGHCROFT INDUSTRIAL ESTATE ENTERPRISE...,116,278,UNIT 29 HIGHCROFT INDUSTRIAL ESTATE ENTERPRISE...,41.7


In [51]:
heavy_100plus_merged.query('PostCode_clean == "EC1V 2NX"')

,PostCode_clean,Address_street,Companies_at_Address,Companies_in_Postcode,FullAddress_best,Address_share_in_PC
115952,EC1V 2NX,124-128 CITY ROAD,8784,9494,"128 CITY ROAD, LONDON, EC1V 2NX",92.5
115976,EC1V 2NX,152-160 CITY ROAD,461,9494,"KEMP HOUSE, 160 CITY ROAD, LONDON, EC1V 2NX",4.9
